# Timelapse 1-count analysis
## Used for estimating gain_model

In [ ]:
# @REMOVE-FROM-TEMPLATE
from plaster.tools.ipynb_helpers.displays import restart_kernel; restart_kernel()

In [ ]:
# Uncomment this to easily reset
# from plaster.tools.ipynb_helpers.displays import restart_kernel; restart_kernel()

In [ ]:
import os
os.environ["MPLCONFIGDIR"] = "/tmp"

In [ ]:
# @IMPORT-MERGE
import numpy as np
import pandas as pd
import itertools
import cv2
import random
from IPython.display import HTML, display
from plaster.tools.log.log import error, debug
from plaster.run.job import JobResult
from plaster.run.run import RunResult
from plaster.run.sigproc_v2 import sigproc_v2_worker as worker
from plaster.run.sigproc_v2.sigproc_v2_result import df_filter, radmat_from_df_filter, df_to_radmat
from plaster.run.plots import plots, plots_dev
from plaster.run.plots.plots_sigproc import plot_psfs, circle_locs, sigproc_v2_im, sigproc_v2_movie_from_df, sigproc_v2_im_from_df
from plaster.run.plots.plots_sigproc import wizard_xy_df, wizard_scat_df, wizard_raw_images
from plaster.run.sigproc_v2.synth import Synth
from plaster.tools.image.coord import WH, XY, roi_shift, clip2d
from plaster.tools.utils import data
from plaster.tools.zplots import zplots
from plaster.tools.schema import check
from plaster.tools.image import imops
from plaster.tools.zap import zap
from plaster.tools.utils import utils
from plaster.tools.utils import data
from plaster.tools.calibration.calibration import Calibration
from plaster.tools.ipynb_helpers.displays import hd, movie
z = zplots.setup()

In [ ]:
# @REMOVE-FROM-TEMPLATE
from plumbum import local
job = JobResult("/erisyon/internal/jobs_folder/abbe6_1t")
run = job.runs[0]

In [ ]:
n_channels = run.sigproc_v2.n_channels
n_cycles = run.sigproc_v2.n_cycles
# dyemat = run.nn_v2._dyemat
# dyepeps = run.nn_v2._dyepeps
# calls_df = run.nn_v2.calls("sigproc")
sigproc_df = run.sigproc_v2.fields__n_peaks__peaks__radmat()
assert n_channels == 1
sig = radmat_from_df_filter(sigproc_df, channel_i=0)

# HACK
sig = sig[:, 0:25]    
    
z.im_clus(sig, _cspan=(-1000, 20000))

# Estimate zero gain model

In [ ]:
# Estimate zero gain by examining the histogram near zero
bins = np.linspace(-2000, 4000, 1000)
_hist, _edges = np.histogram(sig, bins=bins)

# Fit a high degree polynomial to the historgram to get a 
# good estimate of the exact peak
p = np.poly1d(np.polyfit(_edges[1:], _hist, 20))
zero_beta = bins[np.argmax(p(bins))]

# Assume that the samples to the left of this peak are un-contamined
# by signal -- so we can take these, flip the around, and compute
# a standard deviation to get zero_sigma
left = sig[sig < zero_beta]
left = np.concatenate((left, 2*zero_beta-left))
zero_sigma = np.std(left)

# PLOT the results
with z(_merge=True):
    z.hist(sig, _bins=bins, f_x_axis_label="inten", f_title=f"field uncorrected", _size=500)
    z.line(x=[zero_beta, zero_beta], y=[0, np.max(_hist)], color="red")
    z.line(x=[zero_beta-zero_sigma, zero_beta-zero_sigma], y=[0, np.max(_hist)], color="green")
    z.line(x=[zero_beta+zero_sigma, zero_beta+zero_sigma], y=[0, np.max(_hist)], color="green")

dark = zero_beta + 4*zero_sigma
zero_dark = sig.copy()
zero_dark[sig < dark] = 0
z.im_clus(zero_dark, _cspan=(0, 40_000), _size=500)

In [ ]:
# Save for debugging
# name = "abbe8"
# np.save(f"{name}_filt_dyemat", dyemat)
# np.save(f"{name}_filt_radmat.npy", filt_radmat)
# filt_cal_df.to_pickle(f"{name}_filt_calls_df.pkl")
# filt_sig_df.to_pickle(f"{name}_filt_sigproc_df.pkl")